In [23]:
import numpy as np
import scipy
from preprocessing import *
from affichage import *
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import stop_words
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix

In [3]:
path = "data/1___Lost"
corpus = get_corpus(path)

In [38]:
import fnmatch

dico_serie = dict()
liste = []
nb_saisons = sum(os.path.isdir(os.path.join(path, i)) for i in os.listdir(path))
for i in range(1, nb_saisons+1):
    path_saison = path+"/0"+str(i)
    nb_eps_saison = len(fnmatch.filter(os.listdir(path_saison), '*.txt'))
    liste.append(nb_eps_saison)
liste

[24, 24, 24, 13, 20, 20]

In [39]:
l = np.cumsum(liste)
seasons_list = list(range(1, nb_saisons+1))
dico_serie = dict(zip(seasons_list, l))
dico_serie # key : season id value : max index for this season (maxInd+1 actually) in dataframe

{1: 24, 2: 48, 3: 72, 4: 85, 5: 105, 6: 125}

In [9]:
english_contractions_tokens = {'get', 'must', 'this', 'wo', 'may', 'she', 'where', "who'd", 'who', "shouldn't", '(', 'which', "'d", 'I', 'so', 'what', "'ll", 'gim', 'be', 'go', "'re", ')', 's', 'o', 'should', 'something', 'we', "couldn't", "whom'st'd", 'na', 'me', "'t", "mustn't", 'shall', "'m", 'ought', 'sha', 'they', 'why', 'those', 'everyone', 'gon', "e'er", 'it', "ne'er", "'cause", 'ai', 'finna', "'ve", "'s", "'", "o'er", 'could', 'ca', 'might', 'someone', 'somebody', 'these', 'you', "n't", 'da', "o'clock", 'would', 'let', 'there', 'noun', "we'd", 'ol', 'that', 'ta', 'he', "y'all", 'dare', 'need', 'when', 'a', 'how', 'have', 'do'}

In [10]:
sw = {'a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'finds', 'first', 'for', 'four', 'from', 'full', 'fully', 'further', 'furthered', 'furthering', 'furthers', 'g', 'gave', 'general', 'generally', 'get', 'gets', 'give', 'given', 'gives', 'go', 'going', 'good', 'goods', 'got', 'great', 'greater', 'greatest', 'group', 'grouped', 'grouping', 'groups', 'h', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'herself', 'high', 'high', 'high', 'higher', 'highest', 'him', 'himself', 'his', 'how', 'however', 'i', 'if', 'important', 'in', 'interest', 'interested', 'interesting', 'interests', 'into', 'is', 'it', 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kind', 'knew', 'know', 'known', 'knows', 'l', 'large', 'largely', 'last', 'later', 'latest', 'least', 'less', 'let', 'lets', 'like', 'likely', 'long', 'longer', 'longest', 'm', 'made', 'make', 'making', 'man', 'many', 'may', 'me', 'member', 'members', 'men', 'might', 'more', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 'n', 'necessary', 'need', 'needed', 'needing', 'needs', 'never', 'new', 'new', 'newer', 'newest', 'next', 'no', 'nobody', 'non', 'noone', 'not', 'nothing', 'now', 'nowhere', 'number', 'numbers', 'o', 'of', 'off', 'often', 'old', 'older', 'oldest', 'on', 'once', 'one', 'only', 'open', 'opened', 'opening', 'opens', 'or', 'order', 'ordered', 'ordering', 'orders', 'other', 'others', 'our', 'out', 'over', 'p', 'part', 'parted', 'parting', 'parts', 'per', 'perhaps', 'place', 'places', 'point', 'pointed', 'pointing', 'points', 'possible', 'present', 'presented', 'presenting', 'presents', 'problem', 'problems', 'put', 'puts', 'q', 'quite', 'r', 'rather', 'really', 'right', 'right', 'room', 'rooms', 's', 'said', 'same', 'saw', 'say', 'says', 'second', 'seconds', 'see', 'seem', 'seemed', 'seeming', 'seems', 'sees', 'several', 'shall', 'she', 'should', 'show', 'showed', 'showing', 'shows', 'side', 'sides', 'since', 'small', 'smaller', 'smallest', 'so', 'some', 'somebody', 'someone', 'something', 'somewhere', 'state', 'states', 'still', 'still', 'such', 'sure', 't', 'take', 'taken', 'than', 'that', 'the', 'their', 'them', 'then', 'there', 'therefore', 'these', 'they', 'thing', 'things', 'think', 'thinks', 'this', 'those', 'though', 'thought', 'thoughts', 'three', 'through', 'thus', 'to', 'today', 'together', 'too', 'took', 'toward', 'turn', 'turned', 'turning', 'turns', 'two', 'u', 'under', 'until', 'up', 'upon', 'us', 'use', 'used', 'uses', 'v', 'very', 'w', 'want', 'wanted', 'wanting', 'wants', 'was', 'way', 'ways', 'we', 'well', 'wells', 'went', 'were', 'what', 'when', 'where', 'whether', 'which', 'while', 'who', 'whole', 'whose', 'why', 'will', 'with', 'within', 'without', 'work', 'worked', 'working', 'works', 'would', 'x', 'y', 'year', 'years', 'yet', 'you', 'young', 'younger', 'youngest', 'your', 'yours', 'z', 'don', 'yeah', 're', 'okay', 'hey', 'll', 've', 'oh', 'didn', 'ok', 'am', 'yes', 'doin'}

In [14]:
#definition de l'ensemble de stopwords
nltk_sw = set(stopwords.words('english'))
sklearn_sw = set(stop_words.ENGLISH_STOP_WORDS)
stopwords_set = nltk_sw | sklearn_sw | english_contractions_tokens |sw
l_nb = [str(i) for i in range(1000000)]
l_mots = ["don", "yeah", "hey", "okay", "oh", "uh", "yes", "ok"]
for mot in l_mots :
    stopwords_set.add(mot)
for nb in l_nb:
    stopwords_set.add(nb)

In [15]:
vectorizer = TfidfVectorizer(stop_words = stopwords_set, tokenizer=lemmatizing_tokenizer_v2)
X = vectorizer.fit_transform(corpus)
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [25]:
season_1 = tfidf_df[:24]
sparse_matrix_season1 = scipy.sparse.csr_matrix(season_1.values)

In [46]:
dico_Lost = dict()
l = []
minInd = 0
for season, maxInd in dico_serie.items():
    dico_Lost[season] = scipy.sparse.csr_matrix(tfidf_df[minInd:maxInd].values)
    minInd= maxInd
dico_Lost

{1: <24x10817 sparse matrix of type '<class 'numpy.float64'>'
 	with 10625 stored elements in Compressed Sparse Row format>,
 2: <24x10817 sparse matrix of type '<class 'numpy.float64'>'
 	with 9610 stored elements in Compressed Sparse Row format>,
 3: <24x10817 sparse matrix of type '<class 'numpy.float64'>'
 	with 10412 stored elements in Compressed Sparse Row format>,
 4: <13x10817 sparse matrix of type '<class 'numpy.float64'>'
 	with 6376 stored elements in Compressed Sparse Row format>,
 5: <20x10817 sparse matrix of type '<class 'numpy.float64'>'
 	with 10827 stored elements in Compressed Sparse Row format>,
 6: <20x10817 sparse matrix of type '<class 'numpy.float64'>'
 	with 9333 stored elements in Compressed Sparse Row format>}

In [48]:
#version non sparse pour voir
dico_Lost = dict()
l = []
minInd = 0
for season, maxInd in dico_serie.items():
    dico_Lost[season] = tfidf_df[minInd:maxInd].values
    minInd= maxInd
dico_Lost

{1: array([[ 0.        ,  0.03193179,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.04744911,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.03124263,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 2: array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        

In [50]:
len(dico_Lost[1])

24